In [1]:
# | echo: false
# | output: false
export LANG=en_US.UTF-8
export HOME=/Users/joelkim/Work/personal/book_git
cd $HOME

# 인덱스 파일

::: {.hidden}

## 참고 문헌

- Pro Git, Git Internels, Git Objects
  - chrome-extension://oemmndcbldboiebfnladdacbdfmadadm/file:///Users/joelkim/Work/study/study_cs/book/DevOps/Progit/progit.pdf#%5B%7B%22num%22%3A2221%2C%22gen%22%3A0%7D%2C%7B%22name%22%3A%22XYZ%22%7D%2C0%2C510.57%2Cnull%5D
- Mastering Git, Ch.10 Keeping History Clean, An introduction to Git internals
  - chrome-extension://oemmndcbldboiebfnladdacbdfmadadm/file:///Users/joelkim/Work/study/study_cs/book/DevOps/Narebski/2024%20-%20Narebski%20-%20Mastering%20Git,%202nd%20Edition.pdf#p263
- Building Git, Ch.6 The index
  - chrome-extension://oemmndcbldboiebfnladdacbdfmadadm/file:///Users/joelkim/Work/study/study_cs/book/DevOps/Coglan/2021%20-%20Coglan%20-%20Buliding%20Git.pdf#%5B%7B%22num%22%3A524%2C%22gen%22%3A0%7D%2C%7B%22name%22%3A%22XYZ%22%7D%2C72%2C769.889%2Cnull%5D
:::

## git 기본 명령의 내부 구조

- add, commit 등의 명령으로 파일이 등록되는 과정은 내부적으로 다음과 같은 고급 명령을 사용한다.
- 이 중심에는 인덱스 파일이 있다.

| 순번 | git 명령 | 내부 동작 | 설명 |
|-|--|---|------------|
| 1 | git add | 블롭 객체 생성	| `git hash-object` 명령으로 블롭 객체 생성 |
| 2 | - | 인덱스 등록	| `git update-index` 명령으로 인덱스 파일에 스테이징 정보 등록 |
| 3 | git commit | 트리 객체 생성	| `git write-tree` 명령으로 인덱스 파일정보를 이용하여 트리 객체 생성 |
| 4 | - | 커밋 객체 생성 | `git commit-tree` 명령으로 커밋 객체 생성 |
| 5 | - | 브랜치 갱신 | `git update-ref` 명령으로 브랜치 정보 갱신 |

## 실습 1: 레포지토리 준비

- 레포지토리 생성

In [2]:
# | output: false
cd $HOME/lab
rm -rf test_internal_02
git init test_internal_02
cd test_internal_02

Initialized empty Git repository in /Users/joelkim/Work/personal/book_git/lab/test_internal_02/.git/


:::{.cell}
::::{.cell-output .cell-output-stdout}
Initialized empty Git repository in ~/lab/test_internal_02/.git/
::::
:::

In [3]:
tree .git/objects

.git/objects
├── info
└── pack

3 directories, 0 files


## hash-object 명령

- hash-object 명령은 입력된 데이터를 이용하여 해시값을 계산하고 객체를 생성한다.
 - `-w` 옵션을 주지 않으면 단순히 해시값만 계산한다.

```bash
git hash-object [옵션] 입력파일 경로
```

### hash-object 명령의 옵션

- `-t` <유형>
  - 생성할 객체 유형을 지정
  - 기본값: "blob"
  - 가능한 값: "commit", "tree", "blob", "tag"

## 실습 2: 파일의 블롭 객체 생성

In [4]:
echo "file1 line1" > file1.txt

In [5]:
git status

On branch main

No commits yet

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	file1.txt

nothing added to commit but untracked files present (use "git add" to track)


- hash-object 명령으로 file1.txt 파일에 대한 해시값을 계산

In [6]:
git hash-object file1.txt

0b11cfca50e35a4865e8505f1a108bd23a3f9401


- -w 옵션이 없어서 실재로 블롭객체 파일을 생성하지 않음

In [7]:
tree .git/objects

.git/objects
├── info
└── pack

3 directories, 0 files


- -w 옵션을 주면 실재로 블롭객체 파일을 생성함

In [8]:
git hash-object -w file1.txt

0b11cfca50e35a4865e8505f1a108bd23a3f9401


In [9]:
tree .git/objects

.git/objects
├── 0b
│   └── 11cfca50e35a4865e8505f1a108bd23a3f9401
├── info
└── pack

4 directories, 1 file


- cat-file 명령으로 블롭객체의 내용을 읽을 수 있음

In [10]:
git cat-file -p 0b11cfca50e35a4865e8505f1a108bd23a3f9401

file1 line1


## 인덱스 파일

- 인덱스 파일은 다음과 같은 정보를 저장하고 제공하는 바이너리 파일
  - 스테이지와 헤드 커밋에 있는 파일의 메타 정보
    - 파일의 블롭 객체에 대한 포인터
    - 해당 파일의 워크트리 파일 버전의 최신 변경 시간 등에 대한 정보

- add 명령을 실행하면
  - 스테이지에 올라간 파일의 블롭 객체를 생성하고
  - 해당 파일에 대한 메타정보를 인덱스 파일에 등록한다.

## update-index 명령

- hash-object 명령은 객체를 만들 뿐이지 스테이지에 넣지는 않는다.
- 스테이지에 객체를 넣는 것은 update-index 명령을 사용한다.
- update-index 명령은 파일의 메타정보와 내용을 인덱스 파일에 등록한다.
- 실행시에 인덱스 파일이 없으면 새로 생성한다.

### update-index 명령의 옵션

- -add <유형>
  - 인덱스 파일에 파일의 블롭객체를 등록
  - --cacheinfo 옵션이 없으면 블롭객체를 직접 생성함

- `--cacheinfo` <모드> <파일의 블롭객체의 해시값> <파일 경로>
  - 이미 생성된 블롭객체가 존재할 경우 직접 인덱스 파일에 직접 등록

- `--refresh` 
  - 파일 경로 정보를 이용하여 실제 파일에 대해 stat() 시스템 콜을 호출
  - 인덱스 파일의 엔트리 헤더(메타정보)가 stat() 시스템 콜 결과와 다르면 갱신
  - 블롭객체는 건들지 않는다. 

## 실습 3: 인덱스 파일 생성

- 아직 .git/index 파일이 존재하지 않음

In [11]:
ls -al .git

total 24
drwxr-xr-x@  9 joelkim  staff  288 Aug 31 20:04 .
drwxr-xr-x@  4 joelkim  staff  128 Aug 31 20:04 ..
-rw-r--r--@  1 joelkim  staff  137 Aug 31 20:04 config
-rw-r--r--@  1 joelkim  staff   73 Aug 31 20:04 description
-rw-r--r--@  1 joelkim  staff   21 Aug 31 20:04 HEAD
drwxr-xr-x@ 16 joelkim  staff  512 Aug 31 20:04 hooks
drwxr-xr-x@  3 joelkim  staff   96 Aug 31 20:04 info
drwxr-xr-x@  5 joelkim  staff  160 Aug 31 20:04 objects
drwxr-xr-x@  4 joelkim  staff  128 Aug 31 20:04 refs


- 아까 생성한 블롭 객체를 등록

In [12]:
git update-index --add --cacheinfo 100644 0b11cfca50e35a4865e8505f1a108bd23a3f9401 file1.txt

- .git/index 인덱스 파일이 생성됨

In [13]:
ls -al .git

total 32
drwxr-xr-x@ 10 joelkim  staff  320 Aug 31 20:04 .
drwxr-xr-x@  4 joelkim  staff  128 Aug 31 20:04 ..
-rw-r--r--@  1 joelkim  staff  137 Aug 31 20:04 config
-rw-r--r--@  1 joelkim  staff   73 Aug 31 20:04 description
-rw-r--r--@  1 joelkim  staff   21 Aug 31 20:04 HEAD
drwxr-xr-x@ 16 joelkim  staff  512 Aug 31 20:04 hooks
-rw-r--r--@  1 joelkim  staff  104 Aug 31 20:04 index
drwxr-xr-x@  3 joelkim  staff   96 Aug 31 20:04 info
drwxr-xr-x@  5 joelkim  staff  160 Aug 31 20:04 objects
drwxr-xr-x@  4 joelkim  staff  128 Aug 31 20:04 refs


- 인덱스 파일의 내용

In [14]:
hexdump -C .git/index

00000000  44 49 52 43 00 00 00 02  00 00 00 01 00 00 00 00  |DIRC............|
00000010  00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  |................|
00000020  00 00 00 00 00 00 81 a4  00 00 00 00 00 00 00 00  |................|
00000030  00 00 00 00 0b 11 cf ca  50 e3 5a 48 65 e8 50 5f  |........P.ZHe.P_|
00000040  1a 10 8b d2 3a 3f 94 01  00 09 66 69 6c 65 31 2e  |....:?....file1.|
00000050  74 78 74 00 35 4c 42 43  60 45 a6 f6 87 b3 31 4e  |txt.5LBC`E....1N|
00000060  84 73 b8 e3 b9 75 79 c3                           |.s...uy.|
00000068


- 인덱스 항목 중 워크트리의 메타 정보에 해당하는 헤더 정보가 빠져있음. update-index --refresh 명령으로 갱신

In [15]:
git update-index --refresh

In [16]:
hexdump -C .git/index

00000000  44 49 52 43 00 00 00 02  00 00 00 01 68 b4 2c 27  |DIRC........h.,'|
00000010  0d 58 cd a0 68 b4 2c 27  0d 58 cd a0 01 00 00 04  |.X..h.,'.X......|
00000020  09 67 9e e3 00 00 81 a4  00 00 01 f6 00 00 00 14  |.g..............|
00000030  00 00 00 0c 0b 11 cf ca  50 e3 5a 48 65 e8 50 5f  |........P.ZHe.P_|
00000040  1a 10 8b d2 3a 3f 94 01  00 09 66 69 6c 65 31 2e  |....:?....file1.|
00000050  74 78 74 00 c5 4d ba 8b  75 2c 2b 78 d6 91 b5 28  |txt..M..u,+x...(|
00000060  60 63 a7 75 7d 52 89 33                           |`c.u}R.3|
00000068


## 인덱스 파일의 구조

- 구조
  - 인덱스 파일 헤더 (12 바이트)
  - 파일에 대한 인덱스 항목들 (가변)
  - 확장 영역 (가변)
    - 트리 항목 (커밋이 존재하는 경우)
  - 체크섬 (20 바이트)
  

- 인덱스 파일 헤더
  - 총 12 바이트
  - 4바이트 시그너쳐 ('D', 'I', 'R', 'C') ("dircache"를 뜻함)
  - 4바이트 버전숫자 (2~4 중 하나)
  - 32비트 인덱스 엔트리 갯수

- 헤더 예시
  - `44 49 52 43 00 00 00 02  00 00 00 01`
  - DIRC, 버전 2, 1개 엔트리

    ```bash
    00000000  44 49 52 43 00 00 00 02  00 00 00 01 00 00 00 00  |DIRC............|
    ```


- 인덱스 항목 구성
  - 항목 헤더 (파일의 생성일자, 모드, 크기 등 메타정보)
  - 블롭객체 해시값 (파일의 내용)
  - 플래그 + 파일 이름
  - 엔트리 항목 해시값

- 항목 헤더 (40 바이트)
  - 32 비트 ctime 초 (파일 메터메이터가 최종 변경된 시간)
  - 32 비트 ctime 나노초
  - 32 비트 mtime 초 (파일 본문 데이터가 최종 변경된 시간)
  - 32 비트 mtime 나노초
  - 32 비트 dev
  - 32 비트 inode 아이디
  - 32 비트 모드
  - 32 비트 uid
  - 32 비트 gid
  - 32 비트 파일 크기
  ```bash
  00000000  __ __ __ __ __ __ __ __  __ __ __ __ 68 b2 47 74  |            h.Gt|
  00000010  01 6c 0f fd 68 b2 47 74  01 6c 0f fd 01 00 00 04  |.l..h.Gt.l......|
  00000020  09 65 a3 68 00 00 81 a4  00 00 01 f6 00 00 00 14  |.e.h............|
  00000030  00 00 00 0c __ __ __ __  __ __ __ __ __ __ __ __  |....            |
  ```

- 객체 해시값
  - 0b11cfca50e35a4865e8505f1a108bd23a3f9401
  ```bash
  00000030  __ __ __ __ 0b 11 cf ca  50 e3 5a 48 65 e8 50 5f  |........P.ZHe.P_|
  00000040  1a 10 8b d2 3a 3f 94 01  __ __ __ __ __ __ __ __  |....:?.         |
  ```

- 플래그(2 바이트) + 파일 이름(8 바이트 배수로 패딩)
  ```bash
  00000040  __ __ __ __ __ __ __ __  00 09 66 69 6c 65 31 2e  |.       ..file1.|
  00000050  74 78 74 00 __ __ __ __  __ __ __ __ __ __ __ __  |txt.            |
  ```

- 20 바이트 체크섬
  ```bash
  00000050  __ __ __ __ 8c 8e e0 4a  cb ef 63 04 81 44 a6 22  |    ...J..c..D."|
  00000060  8f cd 09 a2 39 cb 45 1c                           |....9.E.|
  ```

### ls-files 명령

- ls-files 명령은 워크트리와 스테이지, 커밋 내부의 파일 목록을 보여주는 명령
- 워크트리의 파일 목록은 실시간으로 검색하지만
- 스테이지와 커밋 파일 목록을 찾을 때 내부적으로 인덱스 파일을 사용함

### ls-files 명령의 옵션

- `-c`
  - 인덱스 파일의 항목 출력
  - 즉, 추적 상태인 파일 목록 출력
  - 아무 옵션이 없는 상태와 동일한 결과

- `--stage`, `-s`
  - 스테이지 영역의 파일만 출력

- `--others`, `-o`
  - 비추적 상태의 파일 목록 출력

- `--ignored`, `-i`
  - 무시되고 있는 파일 목록 출력
  - 반드시 `-o` 옵션과 같이 사용해야 함

- `--debug`
  - 메타 정보까지 출력

## 실습 4: 파일 목록 출력

- 아직 추적되지 않는 신규 파일 생성

In [17]:
mkdir sub
echo "file2 line1" > sub/file2.txt

In [18]:
tree .

.
├── file1.txt
└── sub
    └── file2.txt

2 directories, 2 files


- 추적 상태의 파일 (스테이지 + 커밋)

In [19]:
git ls-files

file1.txt


- 스테이지 파일

In [20]:
git ls-files -s

100644 0b11cfca50e35a4865e8505f1a108bd23a3f9401 0	file1.txt


- 메타 정보 포함

In [21]:
git ls-files --debug

file1.txt
  ctime: 1756638247:223923616
  mtime: 1756638247:223923616
  dev: 16777220	ino: 157785827
  uid: 502	gid: 20
  size: 12	flags: 0


- 비추적 상태의 파일

In [22]:
git ls-files -o

sub/file2.txt


## 실습 5: 다른 파일을 스테이지에 추가

In [23]:
git hash-object -w sub/file2.txt

4a8d995103b41f561e547ce3b32e1983d424da22


In [24]:
tree .git/objects

.git/objects
├── 0b
│   └── 11cfca50e35a4865e8505f1a108bd23a3f9401
├── 4a
│   └── 8d995103b41f561e547ce3b32e1983d424da22
├── info
└── pack

5 directories, 2 files


In [25]:
git update-index --add --cacheinfo 100644 4a8d995103b41f561e547ce3b32e1983d424da22 sub/file2.txt

In [26]:
git update-index --refresh

In [27]:
hexdump -C .git/index

00000000  44 49 52 43 00 00 00 02  00 00 00 02 68 b4 2c 27  |DIRC........h.,'|
00000010  0d 58 cd a0 68 b4 2c 27  0d 58 cd a0 01 00 00 04  |.X..h.,'.X......|
00000020  09 67 9e e3 00 00 81 a4  00 00 01 f6 00 00 00 14  |.g..............|
00000030  00 00 00 0c 0b 11 cf ca  50 e3 5a 48 65 e8 50 5f  |........P.ZHe.P_|
00000040  1a 10 8b d2 3a 3f 94 01  00 09 66 69 6c 65 31 2e  |....:?....file1.|
00000050  74 78 74 00 68 b4 2c 28  37 9b 9d 13 68 b4 2c 28  |txt.h.,(7...h.,(|
00000060  37 9b 9d 13 01 00 00 04  09 67 9e ed 00 00 81 a4  |7........g......|
00000070  00 00 01 f6 00 00 00 14  00 00 00 0c 4a 8d 99 51  |............J..Q|
00000080  03 b4 1f 56 1e 54 7c e3  b3 2e 19 83 d4 24 da 22  |...V.T|......$."|
00000090  00 0d 73 75 62 2f 66 69  6c 65 32 2e 74 78 74 00  |..sub/file2.txt.|
000000a0  00 00 00 00 14 5e 1b 92  d8 ca 60 b7 12 85 02 3f  |.....^....`....?|
000000b0  b6 21 7b 7b 1b 42 43 f2                           |.!{{.BC.|
000000b8


In [28]:
git ls-files -s

100644 0b11cfca50e35a4865e8505f1a108bd23a3f9401 0	file1.txt
100644 4a8d995103b41f561e547ce3b32e1983d424da22 0	sub/file2.txt


In [29]:
git ls-files --debug

file1.txt
  ctime: 1756638247:223923616
  mtime: 1756638247:223923616
  dev: 16777220	ino: 157785827
  uid: 502	gid: 20
  size: 12	flags: 0
sub/file2.txt
  ctime: 1756638248:932945171
  mtime: 1756638248:932945171
  dev: 16777220	ino: 157785837
  uid: 502	gid: 20
  size: 12	flags: 0


## write-tree 명령

- write-tree 명령은 인덱스 내의 항목을 기반으로 커밋 등록에 필요한 트리 객체들을 생성하고 인덱스 파일에 추가
- 생성된 트리 객체 중 루트 트리의 해시값을 출력

## 실습 6: 트리 객체 생성

In [30]:
git write-tree

4c2cf5eb3d8af11e9fe5f56cb6c853e1559d7166


- 2개의 트리 객체가 만들어진다. 

In [31]:
tree .git/objects

.git/objects
├── 0b
│   └── 11cfca50e35a4865e8505f1a108bd23a3f9401
├── 4a
│   └── 8d995103b41f561e547ce3b32e1983d424da22
├── 4c
│   └── 2cf5eb3d8af11e9fe5f56cb6c853e1559d7166
├── dd
│   └── 62677237dce0946aeffef97910ffc4ec32c3e7
├── info
└── pack

7 directories, 4 files


- 각 트리 객체의 내용은 다음과 같다.

In [32]:
git cat-file -p 4c2cf5eb3d8af11e9fe5f56cb6c853e1559d7166

100644 blob 0b11cfca50e35a4865e8505f1a108bd23a3f9401	file1.txt
040000 tree dd62677237dce0946aeffef97910ffc4ec32c3e7	sub


In [33]:
git cat-file -p dd62677237dce0946aeffef97910ffc4ec32c3e7

100644 blob 4a8d995103b41f561e547ce3b32e1983d424da22	file2.txt


- 트리 객체의 경우에는 cat-file -p 명령대신 ls-tree 명령 사용 가능

In [34]:
git ls-tree 4c2cf5eb3d8af11e9fe5f56cb6c853e1559d7166

100644 blob 0b11cfca50e35a4865e8505f1a108bd23a3f9401	file1.txt
040000 tree dd62677237dce0946aeffef97910ffc4ec32c3e7	sub


In [35]:
git ls-tree dd62677237dce0946aeffef97910ffc4ec32c3e7

100644 blob 4a8d995103b41f561e547ce3b32e1983d424da22	file2.txt


- write-tree 명령은 인덱스 파일에도 생성된 트리 객체의 정보를 넣는다 

In [36]:
hexdump -C .git/index

00000000  44 49 52 43 00 00 00 02  00 00 00 02 68 b4 2c 27  |DIRC........h.,'|
00000010  0d 58 cd a0 68 b4 2c 27  0d 58 cd a0 01 00 00 04  |.X..h.,'.X......|
00000020  09 67 9e e3 00 00 81 a4  00 00 01 f6 00 00 00 14  |.g..............|
00000030  00 00 00 0c 0b 11 cf ca  50 e3 5a 48 65 e8 50 5f  |........P.ZHe.P_|
00000040  1a 10 8b d2 3a 3f 94 01  00 09 66 69 6c 65 31 2e  |....:?....file1.|
00000050  74 78 74 00 68 b4 2c 28  37 9b 9d 13 68 b4 2c 28  |txt.h.,(7...h.,(|
00000060  37 9b 9d 13 01 00 00 04  09 67 9e ed 00 00 81 a4  |7........g......|
00000070  00 00 01 f6 00 00 00 14  00 00 00 0c 4a 8d 99 51  |............J..Q|
00000080  03 b4 1f 56 1e 54 7c e3  b3 2e 19 83 d4 24 da 22  |...V.T|......$."|
00000090  00 0d 73 75 62 2f 66 69  6c 65 32 2e 74 78 74 00  |..sub/file2.txt.|
000000a0  00 00 00 00 54 52 45 45  00 00 00 35 00 32 20 31  |....TREE...5.2 1|
000000b0  0a 4c 2c f5 eb 3d 8a f1  1e 9f e5 f5 6c b6 c8 53  |.L,..=......l..S|
000000c0  e1 55 9d 71 66 73 75 62  00 31 20 30 0a dd

## status 명령

- status 명령은 실행시 인덱스 파일을 이용하여 다음과 같은 정보를 알아낸다.
  1. 워크트리의 파일이 추적 상태인지 아닌지 알아낸다.
  2. 추적 상태의 파일인 경우 변경되었는지 아닌지 알아낸다.
  3. 파일이 워크트리, 스테이지, 커밋 중 어느 영역에 있는지 알아낸다. 


- 방법은 다음과 같다.
  - 추적/비추적 여부
    - 인덱스 항목에 없는 파일은 비추적 파일이다.
    - 인덱스 항목에 있는 파일은 스테이지 또는 커밋 영역의 파일이다. 즉, 추적 상태의 파일이다.
  - 추적 파일의 변경 여부
    - 워크트리 파일의 해시값이 인덱스 항목의 블롭 객체 해시값과 같으면 변경되지 않은 파일이다.
    - 워크트리 파일의 해시값이 인덱스 항목의 블롭 객체 해시값과 다르면 변경된 않은 파일이다.
  - 스테이지/커밋 영역의 구별 여부
    - 인덱스 항목의 해시값과 같은 해시값을 가지는 블롭 객체가 커밋에 있으면 커밋 영역의 파일이다.
    - 인덱스 항목의 해시값과 같은 해시값을 가지는 블롭 객체가 커밋에 없으면 스테이지 영역의 파일이다.


## 실습 7: 상태 조회

In [37]:
git status

On branch main

No commits yet

Changes to be committed:
  (use "git rm --cached <file>..." to unstage)
	new file:   file1.txt
	new file:   sub/file2.txt



## commit-tree 명령

- 커밋 객체를 생성

## 실습 8: 커밋 객체 생성

In [38]:
GIT_AUTHOR_DATE="2025-01-01T12:00:00" \
GIT_COMMITTER_DATE="2025-01-01T12:00:00" \
git commit-tree -m 'first commit' 4c2cf5eb3d8af11e9fe5f56cb6c853e1559d7166

e42955ccb9b2b73c7e81e251cbab0fdc84aed671


In [39]:
tree .git/objects

.git/objects
├── 0b
│   └── 11cfca50e35a4865e8505f1a108bd23a3f9401
├── 4a
│   └── 8d995103b41f561e547ce3b32e1983d424da22
├── 4c
│   └── 2cf5eb3d8af11e9fe5f56cb6c853e1559d7166
├── dd
│   └── 62677237dce0946aeffef97910ffc4ec32c3e7
├── e4
│   └── 2955ccb9b2b73c7e81e251cbab0fdc84aed671
├── info
└── pack

8 directories, 5 files


- 커밋 객체는 정상적으로 생성되었음

In [40]:
git cat-file -p e42955ccb9b2b73c7e81e251cbab0fdc84aed671

tree 4c2cf5eb3d8af11e9fe5f56cb6c853e1559d7166
author user <user@company.com> 1735700400 +0900
committer user <user@company.com> 1735700400 +0900

first commit


- 그러나 git status 명령으로는 아직 커밋된 상태가 아니라고 나옴

In [41]:
git status

On branch main

No commits yet

Changes to be committed:
  (use "git rm --cached <file>..." to unstage)
	new file:   file1.txt
	new file:   sub/file2.txt



- 로그도 정상 출력되지 않음

In [42]:
git log || true

fatal: your current branch 'main' does not have any commits yet


## update-ref 명령

- 브랜치가 해당 커밋을 가리키지 않으면 커밋이 생성되었다고 인식하지 않음
- update-ref 명령으로 브랜치가 해당 커밋을 가리키게 만듬

## 실습 9: 브랜치 참조 파일 생성

- HEAD는 main 브랜치를 가리키고 있음

In [43]:
cat .git/HEAD

ref: refs/heads/main


- 그러나 main 브랜치에 대한 참조 파일이 만들어지지 않았음

In [44]:
tree .git/refs/heads

.git/refs/heads

0 directories, 0 files


- update-ref 명령으로 방금 생성한 커밋을 가리키는 main 브랜치 참조 파일 생성

In [45]:
git update-ref refs/heads/main e42955ccb9b2b73c7e81e251cbab0fdc84aed671

- 참조 파일이 생성되었음

In [46]:
tree .git/refs/heads

.git/refs/heads
└── main

1 directory, 1 file


- 참조 파일은 방금 생성된 커밋 객체를 가리킴

In [47]:
cat .git/refs/heads/main

e42955ccb9b2b73c7e81e251cbab0fdc84aed671


- 이제 정상적으로 커밋되었다고 인식함

In [48]:
git status

On branch main
nothing to commit, working tree clean


- 로그도 정상적으로 출력

In [49]:
git log

commit e42955ccb9b2b73c7e81e251cbab0fdc84aed671 (HEAD -> main)
Author: user <user@company.com>
Date:   Wed Jan 1 12:00:00 2025 +0900

    first commit


## 실습 10: add, commit 명령 사용

- 지금까지 한 작업을 add, commit 명령으로 해본다.

- 저장소 리셋

In [50]:
# | output: false
cd $HOME/lab
rm -rf test_internal_02
git init test_internal_02
cd test_internal_02

Initialized empty Git repository in /Users/joelkim/Work/personal/book_git/lab/test_internal_02/.git/


:::{.cell}
::::{.cell-output .cell-output-stdout}
Initialized empty Git repository in ~/lab/test_internal_02/.git/
::::
:::

In [51]:
echo "file1 line1" > file1.txt
mkdir sub
echo "file2 line1" > sub/file2.txt
git add .

In [52]:
hexdump -C .git/index

00000000  44 49 52 43 00 00 00 02  00 00 00 02 68 b4 2c 2d  |DIRC........h.,-|
00000010  24 1a 62 1d 68 b4 2c 2d  24 1a 62 1d 01 00 00 04  |$.b.h.,-$.b.....|
00000020  09 67 9f 31 00 00 81 a4  00 00 01 f6 00 00 00 14  |.g.1............|
00000030  00 00 00 0c 0b 11 cf ca  50 e3 5a 48 65 e8 50 5f  |........P.ZHe.P_|
00000040  1a 10 8b d2 3a 3f 94 01  00 09 66 69 6c 65 31 2e  |....:?....file1.|
00000050  74 78 74 00 68 b4 2c 2d  2a b1 1f 02 68 b4 2c 2d  |txt.h.,-*...h.,-|
00000060  2a b1 1f 02 01 00 00 04  09 67 9f 33 00 00 81 a4  |*........g.3....|
00000070  00 00 01 f6 00 00 00 14  00 00 00 0c 4a 8d 99 51  |............J..Q|
00000080  03 b4 1f 56 1e 54 7c e3  b3 2e 19 83 d4 24 da 22  |...V.T|......$."|
00000090  00 0d 73 75 62 2f 66 69  6c 65 32 2e 74 78 74 00  |..sub/file2.txt.|
000000a0  00 00 00 00 f6 72 a6 22  d9 36 43 3c 2d a0 ad 16  |.....r.".6C<-...|
000000b0  10 37 f6 5e 9f 5b f1 93                           |.7.^.[..|
000000b8


In [53]:
tree .git/objects

.git/objects
├── 0b
│   └── 11cfca50e35a4865e8505f1a108bd23a3f9401
├── 4a
│   └── 8d995103b41f561e547ce3b32e1983d424da22
├── info
└── pack

5 directories, 2 files


In [54]:
GIT_AUTHOR_DATE="2025-01-01T12:00:00" \
GIT_COMMITTER_DATE="2025-01-01T12:00:00" \
git commit -m 'first commit'

[main (root-commit) e42955c] first commit
 2 files changed, 2 insertions(+)
 create mode 100644 file1.txt
 create mode 100644 sub/file2.txt


In [55]:
hexdump -C .git/index

00000000  44 49 52 43 00 00 00 02  00 00 00 02 68 b4 2c 2d  |DIRC........h.,-|
00000010  24 1a 62 1d 68 b4 2c 2d  24 1a 62 1d 01 00 00 04  |$.b.h.,-$.b.....|
00000020  09 67 9f 31 00 00 81 a4  00 00 01 f6 00 00 00 14  |.g.1............|
00000030  00 00 00 0c 0b 11 cf ca  50 e3 5a 48 65 e8 50 5f  |........P.ZHe.P_|
00000040  1a 10 8b d2 3a 3f 94 01  00 09 66 69 6c 65 31 2e  |....:?....file1.|
00000050  74 78 74 00 68 b4 2c 2d  2a b1 1f 02 68 b4 2c 2d  |txt.h.,-*...h.,-|
00000060  2a b1 1f 02 01 00 00 04  09 67 9f 33 00 00 81 a4  |*........g.3....|
00000070  00 00 01 f6 00 00 00 14  00 00 00 0c 4a 8d 99 51  |............J..Q|
00000080  03 b4 1f 56 1e 54 7c e3  b3 2e 19 83 d4 24 da 22  |...V.T|......$."|
00000090  00 0d 73 75 62 2f 66 69  6c 65 32 2e 74 78 74 00  |..sub/file2.txt.|
000000a0  00 00 00 00 54 52 45 45  00 00 00 35 00 32 20 31  |....TREE...5.2 1|
000000b0  0a 4c 2c f5 eb 3d 8a f1  1e 9f e5 f5 6c b6 c8 53  |.L,..=......l..S|
000000c0  e1 55 9d 71 66 73 75 62  00 31 20 30 0a dd

In [56]:
tree .git/objects

.git/objects
├── 0b
│   └── 11cfca50e35a4865e8505f1a108bd23a3f9401
├── 4a
│   └── 8d995103b41f561e547ce3b32e1983d424da22
├── 4c
│   └── 2cf5eb3d8af11e9fe5f56cb6c853e1559d7166
├── dd
│   └── 62677237dce0946aeffef97910ffc4ec32c3e7
├── e4
│   └── 2955ccb9b2b73c7e81e251cbab0fdc84aed671
├── info
└── pack

8 directories, 5 files


## 인덱스 락 파일

- 인덱스 파일에 두 개 이상의 프로세스가 동시에 쓰기를 허용하면 누락 정보가 발생할 수 있음
- 따라서 인덱스 파일에 대한 쓰기 작업은 한 번에 하나의 프로세스만 가능
- 먼저 쓰기를 시도하는 프로세스가 인덱스 파일을 복사한 `index.lock`이라는 인덱스 락 파일을 생성
- 만약 인덱스 락 파일이 존재하면 다른 프로세스는 인덱스 쓰기 실패함
- 쓰기 권한을 가진 프로세스는 인덱스 파일 자체 대신 인덱스 락 파일에 쓰기 시도
- 쓰기가 완료되면 `index.lock` 파일의 이름을 `index`로 바꿔서 한번에 파일 교체

In [57]:
touch .git/index.lock

In [60]:
git add file1.txt || true

fatal: Unable to create '/Users/joelkim/Work/personal/book_git/lab/test_internal_02/.git/index.lock': File exists.

Another git process seems to be running in this repository, e.g.
an editor opened by 'git commit'. Please make sure all processes
are terminated then try again. If it still fails, a git process
may have crashed in this repository earlier:
remove the file manually to continue.


In [61]:
git status

On branch main
nothing to commit, working tree clean
